In [286]:
!pip install torch torchvision
import torch
import torch.nn as nn

In [0]:
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [288]:
from google.colab import drive
drive.mount('/content/gdrive')

class fileLoader():
    
    def fileload(self,PATH):
        x = []
        y = []
        with open(PATH,'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line[:-1]
                temp = line.split(',')
                temp = temp[1:]
                x.append([int(temp[0]),int(temp[1]),int(temp[2]),int(temp[3]),int(temp[4])])
                y.append(int(temp[-1]))
        return x,y

    def fileloadn(self,PATH,n):
        x = []
        y = []
        with open(PATH,'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line[:-1]
                temp = line.split(',')
                if len(temp) != n*5 + 2:
                    continue
                temp2 = []
                for i in temp[1:-1]:
                    temp2.append(int(i))
                temp3 = [temp2[:5],temp2[5:10],temp2[10:15]]
                x.append(temp3)
                y.append(int(temp[-1]))
        return x,y

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
d = fileLoader()
X = [] 
y = []
for i in range(1):
    xt, yt = d.fileloadn('/content/gdrive/My Drive/datas/ndata'+str(i)+'.txt',3)
    X = X + xt
    y = y + yt

In [290]:
print(X)

[[[49350, 49650, 49750, 49000, 15555003], [49850, 49350, 50000, 49100, 13742821], [50100, 49850, 50100, 49300, 15691892]], [[81400, 82000, 82500, 80800, 3182862], [82600, 81700, 83200, 81200, 3221840], [83400, 82500, 83900, 82400, 3185449]], [[591000, 598000, 603000, 588000, 397916], [588000, 593000, 596000, 582000, 174581], [588000, 589000, 598000, 583000, 188425]], [[42500, 42650, 42950, 42500, 1565122], [42750, 42550, 43000, 42250, 1158191], [42400, 42750, 42900, 42400, 1331639]], [[192500, 189000, 197500, 189000, 927247], [197500, 193000, 200000, 192500, 896825], [198000, 197500, 198000, 194500, 433060]], [[211500, 213500, 217000, 210500, 1069375], [212500, 213500, 215000, 211500, 476952], [209000, 212500, 214000, 208000, 718095]], [[353500, 358000, 358500, 350000, 269394], [353500, 353500, 357000, 348000, 442282], [362500, 365000, 371500, 354000, 579674]], [[1434000, 1424000, 1444000, 1406000, 73381], [1395000, 1428000, 1428000, 1394000, 44751], [1371000, 1370000, 1388000, 1362000

In [0]:
class stockSet(Dataset):
    def __init__(self,X,y):
        self.X = np.array(X)
        self.y = np.array(y)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]


In [292]:
for X,y in trainloader:
    print(X.shape,y.shape)
    break

torch.Size([100, 3, 5]) torch.Size([100])


In [0]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [0]:
batch_size = 100
input_dim = 5
hidden_dim = 5
output_dim = 1
num_layers = 1

model = LSTM(input_dim,hidden_dim,output_dim,num_layers,batch_size)
loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [0]:
stock = stockSet(X,y)


In [0]:

#trainloader = torch.cat(trainloader).view(len(trainloader), batch_size, -1)
def train(model, stock, optimizer, loss_fn):
    trainloader = DataLoader(stock, batch_size, shuffle=True, drop_last=True)

    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    train_acc = 0.0
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float()
        y_true = y.float()
        #print(torch.max(X[:, :, 3]), torch.max(y_true))

        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return model, train_loss

model, temp = train(model,stock,optimizer,loss_fn)

In [297]:
print(model,temp)

LSTM(
  (lstm): LSTM(5, 5)
  (regressor): Sequential(
    (0): Linear(in_features=5, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=1, bias=True)
  )
) 1671757184.0


In [0]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)
    loss_fn = torch.nn.MSELoss()

    loss_fn = nn.MSELoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ===================================== #
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        model, train_loss, train_acc = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(model, partition, args)    
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result